## Overview
### Method

### Data download
I downloaded two .zip folders from a Dropbox link; one had raw data files and the other had analysis pipeline files generated by [Mr. DNA Lab](https://www.mrdnalab.com/) Molecular Research. I unzipped the folders and uploaded the Mr. DNA analysis pipeline files into the `coral-pae-temp/analysis/microbiome/mrdna` directory, and the `sample-metadata.tsv` and `demux` folder into `coral-pae-temp/analysis/microbiome/rawdata` directory.  

### Data type

Here I am working with the raw data files located in `coral-pae-temp/analysis/microbiome/rawdata/demux`. In the `demux` folder are two `fastq.gz` files for each of the 22 samples, one for the forward read and one for the reverse read. 

The `fastq.gz` file name includes the sample identifier and should look like `4.Ea_S1_L001_R1_001.fastq.gz`. 
The underscore-separated fields in this file name are:

1.  the sample identifier,

2.  the barcode sequence or a barcode identifier,

3.  the lane number,

4.  the direction of the read (i.e. R1 or R2, because these are paired-end reads), and

5.  the set number.
   

The `fastq.gz` files are **Demultiplexed** (aka **Demuxed**) sequences that still have the forward and reverse primers in the sequences.

-   The Raw Data is **demultiplexed**

-   A R1 and R2 fastq.gz file has been generated for each individual sample

-   All forward reads are binned into the R1 fastq.gz files

-   All reverse reads are binned into the R2 fastq.gz files

-   Other than demultiplexing; you can consider the Raw Data on BaseSpace as untouched (**The Forward and Reverse Primer Sequences have not been removed**)

### Data process
Here I follow the QIIME2 [Casava 1.8 paired-end demultiplexed fastq](https://docs.qiime2.org/2023.5/tutorials/importing/#:~:text=Casava%201.8%20paired%2Dend%20demultiplexed%20fastq) tutorial example on importing data


## Python 3 API import qiime plugins

In [2]:
from qiime2 import Visualization
from qiime2 import Artifact

## Metadata
Make a table of the metadata.
Here I added columns 'Pae', 'Temp', 'PeaTemp', 'Colony', and 'Tank' to the original `sample-metadata.tsv` file provided to me by Mr. DNA and renamed it `sample-metadata-verbose.tsv`
This was a bit of a process... I had to:
upload the `sample-metadata.tsv` to Excel
edit the metadata by adding the above columns and values
save it as a csv
open it in a text editor
search for all ',' commas, and find&replace them with 'TAB' symbols
save as a tab separated file `.tsv`
upload it back into the `coral-pae-temp/analysis/microbiome/rawdata` folder

:::{.callout-important}
At first I had named the new columns 'pae', 'temp', etc. with lower case... for some reason this was a problem later on and the interactive emperor plots wouldn't recognize the new columns. When I changed the column names to CamelCase to match the others, it worked. The `qiime2` docs indicate that metadata formatted with an Identifier Column such as `#Sample ID` is [case-sensitive](https://docs.qiime2.org/2023.5/tutorials/metadata/#metadata-formatting-requirements:~:text=feature%2Did-,Case%2Dsensitive,-(these%20are%20mostly)
:::

In [10]:
!qiime metadata tabulate \
  --m-input-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/blank-mock/metadata-verbose-bm.qzv

Saved Visualization to: ../output/blank-mock/metadata-verbose-bm.qzv


In [12]:
Visualization.load('../output/blank-mock/metadata-verbose-bm.qzv')

<visualization: Visualization uuid: 86523f3b-d618-4590-9c1e-1b6d64e6243a>

Import Sequences into qiime

In [13]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path ../rawdata/demux \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path ../output/blank-mock/demux-paired-end-bm.qza

Imported ../rawdata/demux as CasavaOneEightSingleLanePerSampleDirFmt to ../output/blank-mock/demux-paired-end-bm.qza


The `demux-paired-end.qza` artifact contains raw, demultiplexed sequences that still have forward and reverse primers

## Trim primers from paired-end sequences using `cutadapt`

> "The PCR primers (F515/R806) were developed against the V4 region of the 16S rRNA, which we determined would yield optimal community clustering with reads of this length using a procedure similar to that of ref. 15. [For reference, this primer pair amplifies the region 533–786 in the Escherichia coli strain 83972 sequence (greengenes accession no. prokMSA_id:470367).] The reverse PCR primer is barcoded with a 12-base error correcting Golay code to facilitate multiplexing of up to ≈1,500 samples per lane, and both PCR primers contain sequencer adapter regions." - (Caporasco et al. 2011)

Caporaso, J. G., Lauber, C. L., Walters, W. A., Berg-Lyons, D., Lozupone, C. A., Turnbaugh, P. J., Fierer, N., & Knight, R. (2011). Global patterns of 16S rRNA diversity at a depth of millions of sequences per sample. Proceedings of the National Academy of Sciences, 108(supplement_1), 4516–4522. https://doi.org/10.1073/pnas.1000080107

> "The V4 variable region of the 16S rRNA gene was amplified using the 515F (5′-­GTGCCAGCMGCCGCGGTAA-­3′) and 806R (5′-­GGACTACHVGGGTWTCTAAT-­3′) primer set (Caporaso et al. 2011). ” - (Brown et al. 2021)

Brown, Tanya, Dylan Sonett, Jesse R. Zaneveld, and Jacqueline L. Padilla-Gamiño. 2021. “Characterization of the Microbiome and Immune Response in Corals with Chronic Montipora White Syndrome.” Molecular Ecology 30 (11): 2591–2606. https://doi.org/10.1111/mec.15899.

In [14]:
!qiime cutadapt trim-paired \
  --i-demultiplexed-sequences ../output/blank-mock/demux-paired-end-bm.qza \
  --p-cores 4 \
  --p-front-f GTGYCAGCMGCCGCGGTAA \
  --p-front-r GGACTACNVGGGTWTCTAAT \
  --o-trimmed-sequences ../output/blank-mock/demux-trimmed-bm.qza

Saved SampleData[PairedEndSequencesWithQuality] to: ../output/blank-mock/demux-trimmed-bm.qza


## Visualize trimmed & demultiplexed sequences

In [16]:
!qiime demux summarize \
  --i-data ../output/blank-mock/demux-trimmed-bm.qza \
  --o-visualization ../output/blank-mock/demux-trimmed-summary-bm.qzv

Saved Visualization to: ../output/blank-mock/demux-trimmed-summary-bm.qzv


In [17]:
Visualization.load('../output/blank-mock/demux-trimmed-summary-bm.qzv')

<visualization: Visualization uuid: 9308425d-31c7-4cde-ba9b-742f907bb607>

## [Denoise with DADA2](https://docs.qiime2.org/2023.5/tutorials/moving-pictures/#sequence-quality-control-and-feature-table-construction:~:text=with%20QIIME%201.-,Option%201%3A%20DADA2%C2%B6,-DADA2%20is%20a)

[DADA2](https://pubmed.ncbi.nlm.nih.gov/27214047/) is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. 

As implemented in the q2-dada2 plugin, this quality control process will additionally filter any phiX reads (commonly present in marker gene Illumina sequence data) that are identified in the sequencing data, and will filter chimeric sequences.  
The dada2 denoise-paired method requires four parameters that are used in quality filtering:  

    `--p-trim-left-f m`, which trims off the first m bases of each sequence in the forward reads
    `--p-trim-left-r n`, which trims off the first m bases of each sequence in the reverse reads
    `--p-trunc-len-f o`, which truncates each sequence at position o in the forward reads
    `--p-trunc-len-f p`, which truncates each sequence at position p in the reverse reads  
    
This allows the user to remove low quality regions of the sequences.  

What is a 'good' quality score?  

In QIIME 2's interactive quality plots, the quality scores typically range from 0 to 40. Quality scores reflect the accuracy of base calls in sequencing data, with higher scores indicating higher accuracy. The most common quality score scale used in modern sequencing technologies is the Phred scale.

In the Phred scal: - 

A quality score of 10 corresponds to a 1 in 10 chance of an incorrect base call (90% accurc - y).
A quality score of 20 corresponds to a 1 in 100 chance of an incorrect base call (99% accua - cy).
A quality score of 30 corresponds to a 1 in 1000 chance of an incorrect base call (99.9%acy) 
accur
A "good" quality score in this context depends on your specific analysis goals and the sequencing platform you're u
sing. However, many researchers consider quality scores above 20 to be generally acceptable for downstream anal**ysis. Scores above 30 are often seen as very high qu**
litytheew the Interactive Quality Plot tab in the `demux-trimmed-summary.qzv` file that was generated by `qiime demux summarize`, lots, we see that the quality scores of the bases are high, between a score of 11 in the lowest 2nd percentile and a score of 37 in the bottom 25th percentile and higher. So we won’t trim any bases from the beginning of the sequeI sort of arbitrarily picked 230 as the sequence base position for both the forward and reverse read to truncate.... I'm not sure this is even necessary... a low score of 11 doesn't seem so bad. But, I'm going to keep it in here until I can double check this with someone who better.  

[Denoising tips from Greg Caporasco](https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/020-tutorial-upstream/040-denoising.html) using `qiime dada2 denoise-paired`

 - F, ward- 230
 - R, e31
   
*This next command may take up to 10min to run and is the slowest step*t step

In [18]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs ../output/blank-mock/demux-trimmed-bm.qza \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 230 \
  --p-trunc-len-r 231 \
  --p-n-threads 20 \
  --o-table ../output/blank-mock/table-bm.qza \
  --o-representative-sequences ../output/blank-mock/rep-seqs-bm.qza \
  --o-denoising-stats ../output/blank-mock/denoising-stats-bm.qza

Saved FeatureTable[Frequency] to: ../output/blank-mock/table-bm.qza
Saved FeatureData[Sequence] to: ../output/blank-mock/rep-seqs-bm.qza
Saved SampleData[DADA2Stats] to: ../output/blank-mock/denoising-stats-bm.qza


In [19]:
!qiime metadata tabulate \
  --m-input-file ../output/blank-mock/denoising-stats-bm.qza \
  --o-visualization ../output/blank-mock/denoising-stats-bm.qzv

Saved Visualization to: ../output/blank-mock/denoising-stats-bm.qzv


### Visualize Denoising Stats

In [20]:
Visualization.load('../output/blank-mock/denoising-stats-bm.qzv')

<visualization: Visualization uuid: 930df4f9-2171-4ac4-b487-8cf21e5baa4c>

## Summarize & tabulate the feature table
After the quality filtering step completes, you’ll want to explore the resulting data. You can do this using the following two commands, which will create visual summaries of the data. The `feature-table summarize` command will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics. The `feature-table tabulate-seqs` command will provide a mapping of feature IDs to sequences, and provide links to easily BLAST each sequence against the NCBI nt database.

### feature-table summarize
The feature-table summarize command will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics. Feature tables in QIIME 2 represent the abundance of different biological features (such as bacterial taxa or OTUs) across samples.
 In this command:

--i-table tab.qzal`es the input feature table in QIIME 2 artifact format (.qza file) that you want to sumri

--o-visualizattable.qzv tfies the output visualization in QIIME 2 artifact format (.qzv file) that will con the summaes.

--m-sample-metadata-file sampldata.tsve-cifies the metadata file (usually in tab-separated values format) that contains additional information about the samples in your featble  and m

In [21]:
!qiime feature-table summarize \
  --i-table ../output/blank-mock/table-bm.qza \
  --o-visualization ../output/blank-mock/table-bm.qzv \
  --m-sample-metadata-file ../rawdata/sample-metadata-verbose.tsv

Saved Visualization to: ../output/blank-mock/table-bm.qzv


In [22]:
Visualization.load('../output/blank-mock/table-bm.qzv')

<visualization: Visualization uuid: ee2b6637-f808-40bd-8df0-b1352d5a6679>

### feature-table tabulate-seqs

In [23]:
!qiime feature-table tabulate-seqs \
  --i-data ../output/blank-mock/rep-seqs-bm.qza \
  --o-visualization ../output/blank-mock/rep-seqs-bm.qzv

Saved Visualization to: ../output/blank-mock/rep-seqs-bm.qzv


In [24]:
Visualization.load('../output/blank-mock/rep-seqs-bm.qzv')

<visualization: Visualization uuid: de72ef09-38ca-4d56-ae25-83425c5ed3d3>

## [Generate a tree for phylogenetic diversity analyses](https://docs.qiime2.org/2023.5/tutorials/moving-pictures-usage/#:~:text=Generate%20a%20tree%20for%20phylogenetic%20diversity%20analyses)

From the moving pictures tutorial:
> QIIME supports several phylogenetic diversity metrics, including Faith’s Phylogenetic Diversity and weighted and unweighted UniFrac. In addition to counts of features per sample (i.e., the data in the FeatureTable[Frequency] QIIME 2 artifact), these metrics require a rooted phylogenetic tree relating the features to one another. This information will be stored in a Phylogeny[Rooted] QIIME 2 artifact. To generate a phylogenetic tree we will use align-to-tree-mafft-fasttree pipeline from the q2-phylogeny plugin. 
First, the pipeline uses the mafft program to perform a multiple sequence alignment of the sequences in our FeatureData[Sequence] to create a FeatureData[AlignedSequence] QIIME 2 artifact. Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment. The FastTree program creates an unrooted tree, so in the final step in this section midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

In [25]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences ../output/blank-mock/rep-seqs-bm.qza \
  --output-dir ../output/blank-mock/phylogeny-tree-bm

Saved FeatureData[AlignedSequence] to: ../output/blank-mock/phylogeny-tree-bm/alignment.qza
Saved FeatureData[AlignedSequence] to: ../output/blank-mock/phylogeny-tree-bm/masked_alignment.qza
Saved Phylogeny[Unrooted] to: ../output/blank-mock/phylogeny-tree-bm/tree.qza
Saved Phylogeny[Rooted] to: ../output/blank-mock/phylogeny-tree-bm/rooted_tree.qza


## [Alpha & Beta diversity](https://docs.qiime2.org/2023.5/tutorials/moving-pictures-usage/#:~:text=Alpha%20and%20beta%20diversity%20analysis)

QIIME 2’s diversity analyses are available through the `q2-diversity` plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. We’ll first apply the `core-metrics-phylogenetic` method, which rarefies a `FeatureTable Frequency` to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics. 

The metrics computed by default are:
##### Alpha diversity- 
Shannon’s diversity index (a quantitative measure of community richness- 

Observed Features (a qualitative measure of community richne- s)

Faith’s Phylogenetic Dive**rsity (a qualitative measure of community richness that incorporates phylogenetic relationships between the fea**t- res)

Evenness (or Pielou’s Evenness; a measure of community eve##### nness)

Beta - iversity

Jaccard distance (a qualitative measure of community dis- imilarity)

Bray-Curtis distance (a quantitative measure of community d- ssimilarity)

unweighted U**niFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships betwe**e-  the features)

weighted** UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships bet**ween the features) 


An important parameter that needs to be prii`ded to this scri ` i`s --p-sampling-dep`th, which is the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. For am--ple, if you provide `--p-sampling-depth 5`00, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. Choosing this value is tricky. We recommend making your choice by reviewing the infora`tion prs`ented in th`e table.q`zv file that was created above. Choose a value that is as high as possible (so you retain more sequences per sampl)while excluding as few w samples as possiblefe

### Sampling Depth

In [26]:
## open interactive table visualization
Visualization.load('../output/blank-mock/table-bm.qzv')

## navigate to the interactive sample detail tab
# move the sampling depth slider as high as you can before excluding any samples 
# we want the sampling depth to be high, while retaining all 22 samples
# this looks like a sampling depth of 10,6727 (09AUG2023, SST) 
# but maybe we exclude the mock community and bring it up to 163971? (15AUG2023, SST)

<visualization: Visualization uuid: ee2b6637-f808-40bd-8df0-b1352d5a6679>

What value would you choose to pass for --p-sampling-depth? 
- **104,183**
How many samples will be excluded from your analysis based on this choice? 
- **none, all 22 samples are retained**
How many total reads will you be analyzing in the core-metrics-phylogenetic command?
- **2,292,026**

This represents **42.33%** of the features present across the 22 samples
The mock community has the fewest reads at **104,183** and is our 'limiting factor' to increase sample depth.
Why does the blank have so many features! That is not good... 

### Rooted Tree

In [27]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny ../output/blank-mock/phylogeny-tree-bm/rooted_tree.qza \
  --i-table ../output/blank-mock/table-bm.qza \
  --p-sampling-depth 104183 \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --output-dir ../output/blank-mock/diversity-core-bm

Saved FeatureTable[Frequency] to: ../output/blank-mock/diversity-core-bm/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: ../output/blank-mock/diversity-core-bm/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: ../output/blank-mock/diversity-core-bm/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: ../output/blank-mock/diversity-core-bm/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: ../output/blank-mock/diversity-core-bm/evenness_vector.qza
Saved DistanceMatrix to: ../output/blank-mock/diversity-core-bm/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: ../output/blank-mock/diversity-core-bm/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: ../output/blank-mock/diversity-core-bm/jaccard_distance_matrix.qza
Saved DistanceMatrix to: ../output/blank-mock/diversity-core-bm/bray_curtis_distance_matrix.qza
Saved PCoAResults to: ../output/blank-mock/diversity-core-bm/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: ../ou

### Unweighted Unifrac Emperor Plot

In [28]:
Visualization.load('../output/blank-mock/diversity-core-bm/unweighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: fda23875-f6d6-4be3-ba49-51cc9ccd0e3d>

### Weighted Unifrac Emperor Plot

In [29]:
Visualization.load('../output/blank-mock/diversity-core-bm/weighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 03d37f8b-cf84-40c7-8693-8140b1a2efa8>

### Jaccard Emperor PLot

In [8]:
Visualization.load('../output/diversity-core/jaccard_emperor.qzv')

<visualization: Visualization uuid: 1bb9bae8-4cc0-43da-9a74-de12a9bdd818>

### Bray-Curtis Emperor Plot

In [9]:
Visualization.load('../output/diversity-core/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: 6bffad15-0fbd-4113-8267-eceac7ce61f0>

## Alpha Diversity
After computing diversity metrics, we can begin to explore the microbial composition of the samples in the context of the sample metadata. This information is present in the sample metadata file `../rawdata/sample-metadata-verbose.tsv`.

We’ll first test for associations between categorical metadata columns and alpha diversity data.
We’ll do that here for the Faith Phylogenetic Diversity (a measure of community richness):

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity ../output/diversity-core/faith_pd_vector.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/faith-pd-group-significance.qzv

### Microbial Community Richness: Faith Phylogenetic Diversity

##### 
Which categorical sample metadata columns are most strongly associated with the differences in microbial community richness
Temperature Treatment

##### Are these differences statistically significat?n

Pae
- no significantly different groups

Temp
- ambient (n=12) vs. hot (n=8), pvalue (0.02)
  
PaeTemp
- peak-ambient (n=4) vs. peak-hot (n=4), pvalue (0.04)
- env-ambient (n=4) vs. peak-hot (n=4), pvalue (0.04)

Colony
- no significantly different groups

Tank
- H1 (n=5) vs KB (n=4), pvalue (0.05)
- A2 (n=4) vs H1 (n=5), pvalue (0.05)t?

In [ ]:
Visualization.load('../output/faith-pd-group-significance.qzv')

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity ../output/diversity-core/evenness_vector.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/evenness-group-significance.qzv

### Microbial Community Evenness, Alpha diversity , Pielou's Evenness

##### Which categorical sample metadata columns are most strongly associated with the differences in microbial community evenness? 
Nearly everything but Colony.... 
Surprised by one Tank result: H1 vs H2, I wouldn't have expected those to be different.

##### Are these differences statistically significant?
Pae
- control (n=8) vs peak (n=8), pvalue (0.05)
- env (n=4) vs. peak (n=8), pvalue (0.006)

Temp
- ambient (n=12) vs hot (n=8), pvalue (0.02)

PaeTemp
- control-ambient (n=4) vs peak-ambient (n=4), pvalue (0.02)
- control-ambient (n=4) vs peak-hot (n=4), pvalue (0.02)
- env-ambient (n=4) vs peak-ambient (n=4), pvalue (0.02)
- env-ambient (n=4) vs peak-hot (n=4), pvalue (0.02)
  
Colony
- no significantly different groups

Tank
- A1 (n=4) vs H1 (n=5), pvalue (0.02)
- A1 (n=4) vs KB (n=4), pvalue (0.04)
- A2 (n=4) vs H1 (n=5), pvalue (0.02)
- H1 (n=5) vs H2 (n=3), pvalue (0.05)
- H1 (n=5) vs KB (n=4), pvalue (0.01)

In [ ]:
Visualization.load('../output/evenness-group-significance.qzv')

## Beta Diversity

Next we’ll analyze sample composition in the context of categorical metadata using PERMANOVA (first described in Anderson (2001)) using the beta-group-significance command. The following commands will test whether distances between samples within a group, such as samples from the same body site (e.g., gut), are more similar to each other then they are to samples from the other groups (e.g., tongue, left palm, and right palm). If you call this command with the --p-pairwise parameter, as we’ll do here, it will also perform pairwise tests that will allow you to determine which specific pairs of groups (e.g., tongue and gut) differ from one another, if any. This command can be slow to run, especially when passing --p-pairwise, since it is based on permutation tests. So, unlike the previous commands, we’ll run beta-group-significance on specific columns of metadata that we’re interested in exploring, rather than all metadata columns to which it is applicable. Here we’ll apply this to our unweighted UniFrac distances, using two sample metadata columns, as follows.

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-slim.tsv \
  --m-metadata-column Pae \
  --o-visualization ../output/diversity-core/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-slim.tsv \
  --m-metadata-column Temp \
  --o-visualization ../output/diversity-core/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --m-metadata-column Temp \
  --o-visualization ../output/diversity-core/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix ../output/diversity-core/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/diversity-core/unweighted-unifrac-subject-group-significance.qzv \
  --p-pairwise

## [Atacama Soil Microbiome: Questions to Guide Data Analysis](https://docs.qiime2.org/2023.5/tutorials/atacama-soils/#paired-end-read-analysis-commands:~:text=Questions%20to%20guide%20data%20analysis)
What sample metadata or combinations of sample metadata are most strongly associated with the differences in microbial composition of the samples? Are these associations stronger with unweighted UniFrac or with Bray-Curtis? Based on what you know about these metrics, what does that difference suggest? For exploring associations between continuous metadata and sample composition, the commands qiime metadata distance-matrix in combination with qiime diversity mantel and qiime diversity bioenv will be useful. These were not covered in the Moving Pictures tutorial, but you can learn about them by running them with the `--help` parameter.

## Alpha rarefaction plotting
In this section we’ll explore alpha diversity as a function of sampling depth using the qiime diversity alpha-rarefaction visualizer. This visualizer computes one or more alpha diversity metrics at multiple sampling depths, in steps between 1 (optionally controlled with --p-min-depth) and the value provided as --p-max-depth. At each sampling depth step, 10 rarefied tables will be generated, and the diversity metrics will be computed for all samples in the tables. The number of iterations (rarefied tables computed at each sampling depth) can be controlled with --p-iterations. Average diversity values will be plotted for each sample at each even sampling depth, and samples can be grouped based on metadata in the resulting visualization if sample metadata is provided with the --m-metadata-file parameter.

In [ ]:
!qiime diversity alpha-rarefaction \
  --i-table ../output/dada2-table.qza \
  --i-phylogeny ../output/phylogeny-tree/rooted_tree.qza \
  --p-max-depth 107656 \
  --m-metadata-file ../rawdata/sample-metadata-verbose.tsv \
  --o-visualization ../output/alpha-rarefaction.qzv

In [ ]:
Visualization.load('../output/alpha-rarefaction.qzv')